In [1]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup

In [2]:
#Realicé un webscraping de la página de la Cineteca nacional.
#Para obtener la cartelera del día de hoy, así como la información de cada película. 

In [3]:
#Empezamos obteniendo el código html 
url2 = 'https://www.cinetecanacional.net/controlador.php?opcion=carteleraDia'
cine_link = requests.get(url2).content
cine_soup = BeautifulSoup(cine_link, 'html')
cine_soup

<!DOCTYPE html>
<html>
<head>
<title>Cineteca Nacional MEXICO</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="https://www.cinetecanacional.net/css/general.css" rel="stylesheet" type="text/css"/>
<link href="https://www.cinetecanacional.net/css/mosaico.css" rel="stylesheet" type="text/css"/>
<link href="https://www.cinetecanacional.net/css/cartelera.css" rel="stylesheet" type="text/css"/>
<link href="https://www.cinetecanacional.net/css/style.css" rel="stylesheet" type="text/css"/>
<link href="imagenes/custom-180.png" rel="apple-touch-icon" sizes="152x152"/>
<link href="imagenes/custom-167.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="imagenes/custom-152.png" rel="apple-touch-icon" sizes="167x167"/>
<!--  UTILIZADO PARA EL BUSCADOR DE GOOGLE  
    <link rel="stylesheet" href="http://www.encuentra.gob.mx/css/apf.css" type="text/css">
    <script src="http://www.encuentra.gob.mx/api/gobmxWidgetAPI-min.js" type="text/javascript"></script

In [4]:
#Las películas que tenemos actualmente en la Cineteca Nacional son:
cine_table = cine_soup.find_all('p',{'class':'peliculaTitulo'})
cine_total = [item.text for item in cine_table]
cine_total

[]

In [5]:
#Ahora en bonito en un DataFrame 
df_pelis = pd.DataFrame(cine_total, columns= ['Película'])
df_pelis

,Película


In [6]:
#Ahora obtenemos la minificha de cada película. 
#Tenemos la informacíon del director de cada película, el país de origen, el año y la duración. 
#Aplicamos un find_all  con BeautifulSoup para obtener todo lo referente a la clase 'peliculaMiniFicha'
mini_table = cine_soup.find_all('p',{'class':'peliculaMiniFicha'})
mini_total = [item.text.replace('\r\n', '') for item in mini_table]
mini_total

[]

In [7]:
#Hacemos el split para que nos separe todo eso por comas 
mini_total2 = [item.split(',') for item in mini_total]
mini_total2

[]

In [8]:
#uno = mini_total[0].strip()
#uno_s= uno.split(',')
#uno_s

In [9]:
# Al momento de querer hacer el DataFrame me pedía agregar 2 columnas. 
#Había registros que no tenían solo 4 elementos, sino 6 por lo cuál las rellenaba con 'None'.

mini_df3 = pd.DataFrame(mini_total2, columns = ['Director', 'País', 'Año', 'Duración','0', '1'])
mini_df3

,Director,País,Año,Duración,0,1


In [10]:
#Tenemos 
mini_table = cine_soup.find_all('p',{'class':'peliculaMiniFicha'})
mini_total = [item.text.replace('\r\n', '') for item in mini_table]
mini_total

[]

In [11]:
#Por lo que nadamás sacamos los últimos 4 elementos para que nos mostrara la información que necesitamos. 
#Aplicamos un 'for' para poder iterar columna por columna 
mini_total = [item.split(',') for item in mini_total]
clean_total = [el[-4:] for el in mini_total]
mini_df = pd.DataFrame(clean_total, columns = ['Director', 'País', 'Año', 'Duración'])
mini_df


,Director,País,Año,Duración


In [12]:
#Quitamos los caracteres que no necesitamos como '(' o ')' e imprimimos nuevamente
mini_df['Director'] = mini_df['Director'].str.replace('(', '')
mini_df['Duración'] = mini_df['Duración'].str.replace(')', '')
mini_df['Duración'] = mini_df['Duración'].str.replace('s.', '')
mini_df

,Director,País,Año,Duración


In [13]:
#Sacamos la información de la sinópsis de cada película 
sin_table = cine_soup.find_all('div',{'class':'peliculaImagenSinopsis peliculaSinopsis'})
sin_total = [item.text.replace('\r\n', '') for item in sin_table]
sin_total


[]

In [14]:
#Hacemos el DataFrame 
sin_df = pd.DataFrame(sin_total, columns= ['Sinópsis'])
sin_df['Sinópsis'] = sin_df['Sinópsis'].str.replace('\n', '')
sin_df

,Sinópsis


In [15]:
#Hacemos la concatenación de todos los dataframes para tenerlos todos juntos 
#Al imprimir nos damos cuenta que la sinópsis no está en todas las películas. 
dfs = pd.concat([df_pelis, mini_df, sin_df], axis = 1)
dfs 

,Película,Director,País,Año,Duración,Sinópsis


In [16]:
#Hacemos sólo el dataframe con información de cada película porque algunas películas no tienen sinópsis
dfs2 = pd.concat([df_pelis, mini_df], axis = 1)
dfs2

,Película,Director,País,Año,Duración


In [17]:
#Lo exportamos a un csv para ver si se pudo extrar la información y la muestra limpia 
dfs2.to_csv('cartelera.csv', index=False)